In [289]:
# Approach 1: Find derivative of graph, delete points with too high of a change in slope

import csv, numpy
import plotly.plotly as py
import plotly.graph_objs as graphs
from scipy import interpolate
from scipy.signal import savgol_filter

# Load file into 2d list
with open('../data/P7132033_37.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    zc_str = list(reader)

# Format zc_str to floats and remove low frequency values
zc_x = list()
zc_y = list()
for x, y in zc_str:
    if float(y) > 44000:
        zc_x.append(float(x))
        zc_y.append(float(y))

# Get dy, dy2, and x
prev_y = 0
dy = list()
dy2 = list()
for y in zc_y:
    dy.append(abs(y-prev_y))
    if not len(dy) == 1:
        dy2.append(abs(dy[-1]-dy[-2]))
    prev_y = y
    
# Define cutoff dy for ignoring noise
cutoff = 700
    
# Smooth graph- Savitsky-Golay filter
yhat2 = savgol_filter(zc_y, 27, 2)
yhat3 = savgol_filter(zc_y, 17, 3)
yhat4 = savgol_filter(zc_y, 17, 4) #best fit so far

# ELIMINATE NOISE
i = 0
noiseless_x = list()
noiseless_y = list()
while i < len(dy2):
    if dy2[i] < cutoff:
        noiseless_x.append(zc_x[i])
        noiseless_y.append(yhat4[i])
    i += 1

# Graph data
trace_noisy = graphs.Scatter(
                            x = zc_x, 
                            y = zc_y,
                            mode = 'markers',
                            name = 'ZC- noisy'
                        )
trace_smooth = graphs.Scatter(
                            x = zc_x, 
                            y = yhat4,
                            name = 'ZC- smoothed'
                        )
trace_noiseless = graphs.Scatter(
                            x = noiseless_x, 
                            y = noiseless_y,
                            mode = 'markers',
                            name = 'ZC- noiseless'
                         )
trace_dy2 = graphs.Scatter(
                            x = zc_x, 
                            y = dy2, 
                            name = 'Derivative- noisy'
                         )
trace_cutoff = graphs.Scatter(
                            x = [zc_x[0], zc_x[-1]], 
                            y = [cutoff, cutoff], 
                            name = 'Cutoff'
                         )
trace = [trace_noisy, trace_smooth, trace_noiseless, trace_cutoff, trace_dy2]
py.iplot(trace, filename='P7132033_37')

In [295]:
# Approach 2: Smooth graph, and take comparisons of smoothed and unsmoothed graph. More different points are deleted

import csv, numpy
import plotly.plotly as py
import plotly.graph_objs as graphs
from scipy import interpolate
from scipy.signal import savgol_filter

# Load file into 2d list
with open('../data/P7132033_37.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    zc_str = list(reader)

# Format zc_str to floats and remove low frequency values
zc_x = list()
zc_y = list()
for x, y in zc_str:
    if float(y) > 44000:
        zc_x.append(float(x))
        zc_y.append(float(y))
    
# Define cutoff dy for ignoring noise
cutoff = 300
    
# Smooth graph- Savitsky-Golay filter
yhat2 = savgol_filter(zc_y, 27, 2)
yhat3 = savgol_filter(zc_y, 17, 3)
yhat4 = savgol_filter(zc_y, 7, 4) # best fit so far

# Compare smoothed and original
i = 0
noiseless_y = list()
noiseless_x = list()
dy = list()
while i < len(zc_x):
    dy.append(abs(zc_y[i] - yhat4[i]))
    if dy[-1] < cutoff:
        noiseless_y.append(zc_y[i])
        noiseless_x.append(zc_x[i])
    i += 1
    
# Graph data
trace_noisy = graphs.Scatter(
                            x = zc_x, 
                            y = zc_y,
                            mode = 'markers',
                            name = 'ZC- noisy'
                        )
trace_smooth = graphs.Scatter(
                            x = zc_x, 
                            y = yhat4,
                            name = 'ZC- smoothed'
                        )
trace_noiseless = graphs.Scatter(
                            x = noiseless_x, 
                            y = noiseless_y,
                            mode = 'markers',
                            name = 'ZC- noiseless'
                         )
trace_dy = graphs.Scatter(
                            x = zc_x, 
                            y = dy, 
                            name = 'Derivative- noisy'
                         )
trace_cutoff = graphs.Scatter(
                            x = [zc_x[0], zc_x[-1]], 
                            y = [cutoff, cutoff], 
                            name = 'Cutoff'
                         )
trace = [trace_noisy, trace_smooth, trace_noiseless, trace_cutoff, trace_dy]
py.iplot(trace, filename='P7132033_37')

In [357]:
# Approach 3: Smooth graph, and take comparisons of smoothed and unsmoothed graph. Go by chunks and take average difference between smooth and unsmoothed.

import csv, numpy
import plotly.plotly as py
import plotly.graph_objs as graphs
from scipy import interpolate
from scipy.signal import savgol_filter

# Magic numbers
cutoff = 50
avg_dx = 0.0002
pulse_size = 15

# Load file into 2d list
with open('../data/P7132033_37.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    zc_str = list(reader)

# Format zc_str to floats and remove low frequency values
zc_x = list()
zc_y = list()
for x, y in zc_str:
        zc_x.append(float(x))
        zc_y.append(float(y))
    
# Smooth graph- Savitsky-Golay filter
yhat2 = savgol_filter(zc_y, 27, 2)
yhat3 = savgol_filter(zc_y, 17, 3)
yhat4 = savgol_filter(zc_y, 17, 4) # best fit so far

# Compare smoothed and original
i = 0
noiseless_y = list()
noiseless_x = list()
dy = list()
while i < len(zc_x):
    j = i
    average = 0
    while j < len(zc_x) and (numpy.sqrt((zc_x[j] - zc_x[j - 1])**2 + (zc_y[j] - zc_y[j - 1])**2)) <= 1500:
        average += abs(zc_y[i] - yhat4[i])
        j += 1
    if j - i > pulse_size and average / (j - i) <= cutoff:
        while i < j:
            noiseless_y.append(zc_y[i])
            noiseless_x.append(zc_x[i])
            i += 1
    i += 1
    
# Graph data
trace_noisy = graphs.Scatter(
                            x = zc_x, 
                            y = zc_y,
                            mode = 'markers',
                            name = 'ZC- noisy'
                        )
trace_smooth = graphs.Scatter(
                            x = zc_x, 
                            y = yhat4,
                            name = 'ZC- smoothed'
                        )
trace_noiseless = graphs.Scatter(
                            x = noiseless_x, 
                            y = noiseless_y,
                            mode = 'markers',
                            name = 'ZC- noiseless'
                         )
trace = [trace_noisy, trace_smooth, trace_noiseless]
py.iplot(trace, filename='P7132033_37')

In [314]:
import csv, numpy
import plotly.plotly as py
import plotly.graph_objs as graphs
from scipy import interpolate
from scipy.signal import savgol_filter

# Load file into 2d list
with open('../data/P7132033_37.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    zc_str = list(reader)

# Format zc_str to floats and remove low frequency values
zc_x = list()
zc_y = list()
for x, y in zc_str:
    if float(y) > 44000:
        zc_x.append(float(x))
        zc_y.append(float(y))
        
i = 0
j = 0
average = 0
while i < len(zc_x):
    if abs(zc_x[i - 1] - zc_x[i]) < 0.004243201857585126:
        average += abs(zc_x[i - 1] - zc_x[i])
        j += 1
        
    i += 1
print (average/j)

0.00015388240827845712
